In [179]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile
import math

from datetime import datetime
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [180]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize, num):
        self.x, self.y, self.batchsize, self.num = x, y, batchsize, num

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def names_at_batch(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        return x_names, y_names

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])

        # open x image names, resize, normalise and make a numpy array
        x = np.asarray([preprocess_input(img_to_array(load_img(file_name[self.num], target_size=(299, 299)))) for file_name in x_names])

        return x, y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [181]:
class ValGenerator(SequenceGenerator):
    def __init__(self, x, y, batchsize):
        super().__init__(x, y, batchsize, 0)
        x1 = np.asarray([file_name[0] for file_name in self.x])
        x2 = np.asarray([file_name[1] for file_name in self.x])
        x3 = np.asarray([file_name[2] for file_name in self.x])
        self.x = np.concatenate((x1, x2, x3))

        self.y = np.concatenate((self.y, self.y, self.y)).astype(int)

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])

        x = np.asarray([preprocess_input(img_to_array(load_img(file_name, target_size=(299, 299)))) for file_name in x_names])

        return x, y_names

In [182]:
def get_filenames_from_dir(directory):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        triplet = []
        if isfile(category):
            continue
        for file in listdir("{0}/{1}".format(directory, category)):
            if file[-3:] != "jpg":
                continue
            triplet.append(("{0}/{1}/{2}".format(directory, category, file), category_count))
            if len(triplet) == 3:
                x.append([img[0] for img in triplet])
                y.append(triplet[0][1])
                triplet = []
        category_count += 1

    return x, y

In [183]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database")

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

In [184]:
# make generators with batch size 32 for each set
train_gen1 = SequenceGenerator(x_train, y_train, 4, 0)
train_gen2 = SequenceGenerator(x_train, y_train, 4, 1)
train_gen3 = SequenceGenerator(x_train, y_train, 4, 2)
val_gen = ValGenerator(x_val, y_val, 4)
test_gen = ValGenerator(x_test, y_test, 4)

In [185]:
# Model 1
base_model1 = InceptionV3(weights="imagenet")
for layer in base_model1.layers:
    layer._name += "_1"
    layer.trainable = True
predictions1 = keras.layers.Dense(291, activation='softmax')(base_model1.layers[-2].output)
model1 = keras.Model(inputs=base_model1.input, outputs=predictions1)
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model 2
base_model2 = InceptionV3(weights="imagenet")
for layer in base_model2.layers:
    layer._name += "_2"
    layer.trainable = True
predictions2 = keras.layers.Dense(291, activation='softmax')(base_model2.layers[-2].output)
model2 = keras.Model(inputs=base_model2.input, outputs=predictions2)
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model 3
base_model3 = InceptionV3(weights="imagenet")
for layer in base_model3.layers:
    layer._name += "_3"
    layer.trainable = True
predictions3 = keras.layers.Dense(291, activation='softmax')(base_model3.layers[-2].output)
model3 = keras.Model(inputs=base_model3.input, outputs=predictions3)
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [186]:
logdir1 = "logs/3model_individual_unshared/model1_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback1 = keras.callbacks.TensorBoard(log_dir=logdir1)

logdir2 = "logs/3model_individual_unshared/model2_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback2 = keras.callbacks.TensorBoard(log_dir=logdir2)

logdir3 = "logs/3model_individual_unshared/model3_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback3 = keras.callbacks.TensorBoard(log_dir=logdir3)

In [187]:
model1.fit(train_gen1, validation_data=val_gen, callbacks=[tensorboard_callback1], epochs=20)
print("Model 1 completed\n\n")
model2.fit(train_gen2, validation_data=val_gen, callbacks=[tensorboard_callback2], epochs=20)
print("Model 2 completed\n\n")
model3.fit(train_gen3, validation_data=val_gen, callbacks=[tensorboard_callback3], epochs=20)
print("Model 3 completed")

Epoch 1/20
3709/3709 [==============================] - 331s 87ms/step - loss: 5.4850 - accuracy: 0.0147 - val_loss: 5.3287 - val_accuracy: 0.0179
Epoch 2/20
3709/3709 [==============================] - 312s 84ms/step - loss: 5.0547 - accuracy: 0.0280 - val_loss: 5.9792 - val_accuracy: 0.0321
Epoch 3/20
3709/3709 [==============================] - 309s 83ms/step - loss: 4.6365 - accuracy: 0.0473 - val_loss: 4.6945 - val_accuracy: 0.0342
Epoch 4/20
3709/3709 [==============================] - 309s 83ms/step - loss: 4.2164 - accuracy: 0.0760 - val_loss: 4.1810 - val_accuracy: 0.0850
Epoch 5/20
3709/3709 [==============================] - 309s 83ms/step - loss: 3.7337 - accuracy: 0.1325 - val_loss: 4.8716 - val_accuracy: 0.0646
Epoch 6/20
3709/3709 [==============================] - 310s 84ms/step - loss: 3.1952 - accuracy: 0.2095 - val_loss: 3.9616 - val_accuracy: 0.1419
Epoch 7/20
3709/3709 [==============================] - 309s 83ms/step - loss: 2.6943 - accuracy: 0.2932 - val_loss: 3

In [188]:
model1.evaluate(test_gen)
model2.evaluate(test_gen)
model3.evaluate(test_gen)

2366/2366 [==============================] - 61s 26ms/step - loss: 2.3032 - accuracy: 0.3908


[2.3031606674194336, 0.3908264636993408]